In [1]:
import torch

In [2]:
torch.__version__

'2.0.1'

In [3]:
import os
import pytorch_lightning as pl
import torch

from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

C:\Users\kyeul\anaconda3\envs\nlp\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [5]:
try:
    from torchmetrics.functional import accuracy
    print("Import torchmetrics")
except ImportError:
    from pytorch_lightning.metrics.functional import accuracy
    print("Import pytorch_lightning")

Import torchmetrics


In [6]:
import mlflow.pytorch
from mlflow import MlflowClient

In [38]:
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)
        
    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        logits = self(x) # 이건 뭐지?
        loss = F.cross_entropy(logits, y) 
        pred = logits.argmax(dim=1)
        acc = accuracy(pred, y, task="multiclass", num_classes=10)
        
        self.log("train_loss", loss, on_epoch=True)
        self.log("acc", acc, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [8]:
def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print(f"run_id: {r.info.run_id}")
    print(f"artifacts: {artifacts}")
    print(f"params: {r.data.params}")
    print(f"metrics: {r.data.metrics}")
    print(f"tags: {tags}")

In [39]:
# 모델 초기화
mnist_model = MNISTModel()

In [10]:
# MNIST데이터셋으로부터 Dataloader 초기화
train_ds = MNIST(
    os.getcwd(), train=True, download=True, transform=transforms.ToTensor()
)


100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:01<00:00, 6501790.33it/s]


Extracting C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw\train-images-idx3-ubyte.gz to C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 5731520.88it/s]


Extracting C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw\train-labels-idx1-ubyte.gz to C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw



100%|██████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 16964809.97it/s]


Extracting C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw\t10k-images-idx3-ubyte.gz to C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw\t10k-labels-idx1-ubyte.gz to C:\Users\kyeul\Desktop\code\practice_language\python\ML\MLFlow\MNIST\raw



In [40]:
train_loader = DataLoader(train_ds, batch_size=32)

In [41]:
trainer = pl.Trainer(max_epochs=20)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [42]:
# Auto log all MLflow entities
mlflow.pytorch.autolog()

In [43]:
# Train the model
with mlflow.start_run() as run:
    trainer.fit(mnist_model, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Epoch 19: 100%|███████████████████████████████████████████████████████████| 1875/1875 [00:34<00:00, 54.49it/s, v_num=3]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|███████████████████████████████████████████████████████████| 1875/1875 [00:34<00:00, 54.43it/s, v_num=3]


2023/10/04 23:36:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\kyeul\anaconda3\envs\nlp\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [44]:
# fetch the auto logged parameters and metrics
print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))

run_id: ae998dbe58d44289924b31d08c9d764e
artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/data', 'model/python_env.yaml', 'model/requirements.txt']
params: {'amsgrad': 'False', 'betas': '(0.9, 0.999)', 'capturable': 'False', 'differentiable': 'False', 'epochs': '20', 'eps': '1e-08', 'foreach': 'None', 'fused': 'None', 'lr': '0.02', 'maximize': 'False', 'optimizer_name': 'Adam', 'weight_decay': '0'}
metrics: {'acc': 0.5550666451454163, 'acc_epoch': 0.5550666451454163, 'acc_step': 0.625, 'train_loss': 1.1148642301559448, 'train_loss_epoch': 1.1148642301559448, 'train_loss_step': 0.8634935021400452}
tags: {'Mode': 'training'}
